# AI Cup - Command notebook 🏆

### Looked at

- Brute force
- DP
- LKH
- concorde
- ACO
- christofides
- Simulated Annealing


### Useful assets

https://www2.seas.gwu.edu/~simhaweb/champalg/tsp/tsp.html

http://160592857366.free.fr/joe/ebooks/ShareData/Heuristics%20for%20the%20Traveling%20Salesman%20Problem%20By%20Christian%20Nillson.pdf

https://science.lpnu.ua/sites/default/files/journal-paper/2017/may/2487/pubv7eng.pdf

https://github.com/istresec/kth-aa/blob/main/report.pdf

### Global variables

In [11]:
debug = True
SEED = 69

### Modules needed for Trainer to manage

In [12]:
import glob
import random
import math
import subprocess
import time
import numpy as np
import pandas as pd
from tqdm import trange, tqdm
from matplotlib import pyplot as plt
from IPython.display import display, clear_output

### Get all files loaded. Check if missing.

In [13]:
problems = glob.glob('./problems/*.tsp')
print("[Success] All files loaded" if np.all([n in ['./problems/fl1577.tsp','./problems/pr439.tsp','./problems/ch130.tsp','./problems/rat783.tsp','./problems/d198.tsp', './problems/kroA100.tsp','./problems/u1060.tsp','./problems/lin318.tsp','./problems/eil76.tsp','./problems/pcb442.tsp'] for n in problems]) else "[Error] Missing files")

[Success] All files loaded


### View problems

In [14]:
import nn
problems_frame = []
for problem in problems:
    with open(problem,"r") as probfile:
        file = probfile.read().splitlines()
        name = file[0].split(": ")[1]
        points = int(file[3].split(": ")[1])
        _nn = nn.run(problem)
        best = file[5].split(": ")[1]

    problems_frame.append([name, points, _nn, best])     

problems_frame = pd.DataFrame(problems_frame, columns=["name", "points", "nn", "best"])
problems_frame

,name,points,nn,best
0,lin318,318,54019.0,42029
1,pr439,439,137481.0,107217
2,rat783,783,10754.0,8806
3,kroA100,100,27807.0,21282
4,eil76,76,680.0,538
5,d198,198,18975.0,15780
6,u1060,1060,309386.0,224094
7,fl1577,1577,28498.0,22249
8,ch130,130,7578.0,6110
9,pcb442,442,61867.0,50778


# T_Executor 🎯 
### Execute a training call

In [15]:
def t_execute(call):
    start = time.time()
    output = subprocess.run(call, stdout=subprocess.PIPE)
    end = time.time()
    output = output.stdout.decode('utf-8').splitlines()
    duration = round(end - start, 2)
    output = output[-1].split(" - ")
    path = int(output[0])
    signal = output[1]
    avg_run = round(float(output[2]),5)
    avg_iiopt = float(output[3])
    score = str(round(((path-best)/best)*100,2))
    #print(f"[Success] Name: {name} - Points: {points} - Path: {path} - Signal: {signal} - Time: {duration}s - AvgRun: {avg_run} - AvgIIopt: {avg_iiopt} - Score: {score}%")
    return [name, points, path, signal, duration, avg_run, avg_iiopt, score]

# Training 🏋🏻‍♂️

### Hyperparameter tuning. Train for hyperparameters, time and seeds. Find the optimal combination of all.

In [16]:
%%script false --no-raise-error

# try out of time to calculate average iterations
import itertools
problems_frame_s = problems_frame.sort_values(by=['points'])
problems_frame_s.reset_index(drop=True, inplace=True)
args = []
args.append(["./aco_2opt_wc", str(120), "./problems/eil76.tsp", str(SEED), str(500), str(50), str(4), str(1), str(0.8), str(0.1), None])
args.append(["./aco_2opt_wc", str(120), "./problems/kroA100.tsp", str(SEED), str(500), str(50), str(2), str(3), str(0.9), str(0.1), None])
args.append(["./aco_2opt_wc", str(120), "./problems/ch130.tsp", str(SEED), str(500), str(130), str(2), str(3), str(0.99), str(0.1), None])
args.append(["./aco_2opt_wc", str(120), "./problems/d198.tsp", str(SEED), str(500), str(100), str(2), str(3), str(0.9), str(0.2), None])
args.append(["./aco_2opt_wc", str(120), "./problems/lin318.tsp", str(SEED), str(500), str(50), str(3), str(2), str(0.99), str(0.1), None])
args.append(["./aco_2opt_wc", str(120), "./problems/pr439.tsp", str(SEED), str(500), str(100), str(4), str(2), str(0.99), str(0.1), None])
args.append(["./aco_2opt_wc", str(120), "./problems/pcb442.tsp", str(SEED), str(500), str(100), str(2), str(3), str(0.9), str(0.1), None])
args.append(["./aco_2opt_wc", str(120), "./problems/rat783.tsp", str(SEED), str(500), None, str(2), str(3), str(0.9), str(0.1), None])
args.append(["./aco_2opt_wc", str(120), "./problems/u1060.tsp", str(SEED), str(500), None, str(2), str(3), str(0.9), str(0.1), None])
args.append(["./aco_2opt_wc", str(120), "./problems/fl1577.tsp", str(SEED), str(500), None, str(2), str(3), str(0.9), str(0.5), None])
for index, row in problems_frame_s.iterrows():
    # Info
    points = int(row['points'])
    if index > 4:
        points = points/2
    _nn = int(row['nn'])
    tau0 = 1/(points*_nn)
    args[index][5] = str(points)
    args[index][10] = str(tau0)

for index, row in problems_frame_s.iterrows():
    # Info
    name = row['name']
    points = int(row['points'])
    if index > 4:
        points = points/2
    best = int(row['best'])

    # Paramenters
    ac = [points, points*0.75, points*0.5, points*0.25, points*0.125]
    alpha = [1, 2, 3, 4]
    beta = [1, 2, 3, 4]
    ab_combo = list(itertools.product(alpha, beta))
    rho = [0.99, 0.95, 0.9, 0.8]
    q0 = [0.1, 0.2, 0.3, 0.4]
    rq_combo = list(itertools.product(rho, q0))

    # Save
    test_problems_executed = []

    # Run
    my_call = args[index]
    exec = t_execute(my_call)
    exec.append(" ".join(my_call))
    test_problems_executed.append(exec)

    best_score = float(exec[7][:-1])

    for _ in trange(2, desc=name):
        for a in ac:
            call = my_call.copy()
            call[5] = str(a)
            exec = t_execute(call)
            exec.append(" ".join(call))
            test_problems_executed.append(exec)
            score = float(exec[7][:-1])
            if score < best_score:
                best_score = score
                my_call = call
            print(f"Best gap: {best_score}%", sep=' ', end='\r', flush=True)

        for a, b in ab_combo:
            call = my_call.copy()
            call[6] = str(a)
            call[7] = str(b)
            exec = t_execute(call)
            exec.append(" ".join(call))
            test_problems_executed.append(exec)
            score = float(exec[7][:-1])
            if score < best_score:
                best_score = score
                my_call = call
            print(f"Best gap: {best_score}%", sep=' ', end='\r', flush=True)

        for r, q in rq_combo:
            call = my_call.copy()
            call[8] = str(r)
            call[9] = str(q)
            exec = t_execute(call)
            exec.append(" ".join(call))
            test_problems_executed.append(exec)
            score = float(exec[7][:-1])
            if score < best_score:
                best_score = score
                my_call = call
            print(f"Best gap: {best_score}%", sep=' ', end='\r', flush=True)

    # Save
    test_problems_executed = pd.DataFrame(test_problems_executed, columns=["name", "points", "path", "signal", "time", "avg_run", "avg_iiopt", "score", "call_str"])
    # test_problems_executed.to_csv(f"./results/{name}.csv", index=False)
    

# Running on Trained 🚀
### Executing already built modules. Make sure to run Training First.

In [17]:
files = glob.glob("results/*.csv")
scores = []
for file in files:
    df = pd.read_csv(file)
    this_score = df["score"].min()
    rows = df.loc[df["score"] == this_score]
    this_call = rows.iloc[-1]["call_str"]
    this_call = this_call.split(" ")
    scores.append(
        {
            "command": this_call[0],
            "time": this_call[1],
            "file": this_call[2],
            "seed": this_call[3],
            "iterations": this_call[4],
            "points": this_call[5],
            "alpha": this_call[6],
            "beta": this_call[7],
            "rho": this_call[8],
            "q0": this_call[9],
            "tau0": this_call[10],
        }
        )

scores = pd.DataFrame(scores, columns=["command", "time", "file", "seed", "iterations", "points", "alpha", "beta", "rho", "q0", "tau0"])
# scores.to_csv("command.csv", index=False)

# R_Executor 🎯 
### Execute a final call. Tweak the command.csv for best results.

In [18]:
def r_execute(name, best, call):
    start = time.time()
    output = subprocess.run(call, stdout=subprocess.PIPE)
    end = time.time()
    output = output.stdout.decode('utf-8').splitlines()
    duration = round(end - start, 2)
    output = output[-1].split(" - ")
    path = int(output[0])
    score = str(round(((path-best)/best)*100,2))
    signal = output[1]
    avg_run = round(float(output[2]),5)
    if call[0] != "./aco_3opt_wc":
        avg_2opt = float(output[3])
    else:
        avg_2opt = output[3]
    if call[0] != "./aco_2opt_wc":
        opt3 = float(output[4])
    else:
        opt3 = "None"
    #print(f"[Success] Name: {name} - Points: {points} - Path: {path} - Signal: {signal} - Time: {duration}s - AvgRun: {avg_run} - AvgIIopt: {avg_iiopt} - Score: {score}%")
    return {
        "command": call[0],
        "name": name,
        "path": path,
        "signal": signal,
        "time": str(duration) + "s",
        "avg_cycle": avg_run,
        "avg_2opt": avg_2opt,
        "3opt": opt3,
        "score": score,
    }

In [19]:
def get_from_trained(name):
    training_set = pd.read_csv(f"results/{name}.csv")
    best = training_set["path"].min()
    best_row = training_set.loc[training_set["path"] == best]
    best_row = best_row.iloc[-1]
    best_row = best_row

    print(best_row)
    return {
        "command": best_row["call_str"].split(" ")[0],
        "name": name,
        "path": best,
        "signal": best_row["signal"],
        "time": str(best_row["time"])+"s",
        "avg_cycle": best_row["avg_run"],
        "avg_2opt": best_row["avg_iiopt"],
        "3opt": "None",
        "score": best_row["score"],

    }

# Build results 🏗
### Build the final.csv file either running from command csv or from a trained instance.

In [ ]:
command = pd.read_csv("command_final.csv")

# run
runs = []
for index, row in command.iterrows():
    call = [
        row["command"],
        str(row["time"]),
        row["file"],
        str(row["seed"]),
        str(row["iterations"]),
        str(row["points"]),
        str(row["alpha"]),
        str(row["beta"]),
        str(row["rho"]),
        str(row["q0"]),
        str(row["tau0"]),
    ]
    name = row["file"].strip(".tsp")[10:]
    best = int(problems_frame.loc[problems_frame["name"] == name]["best"].values[0])
    if name == "None":
        values = get_from_trained(name)
    else:
        values = r_execute(name, best, call)
    runs.append(values)
    clear_output(wait=True)
    display(pd.DataFrame(runs))
# pd.DataFrame(runs).to_csv("final.csv", index=False)